In [15]:
import pandas as pd
import requests

def store_nationalities(query_results, path_to_nationalities_csv):
    """Creates a csv listing all Carnegie composer IDs and all their nationalities, which are stored in a string."""
    nationalities_df = pd.read_csv('CarnegieData/name_skos.csv')
    nationalities_df.insert(2, 'nationalities', pd.Series(dtype=str))
    
    for index in nationalities_df.index:
            # isolate the data in the 'skos' row
            wikidata_id = nationalities_df.loc[index, 'skos']
            # isolate the unique wikidata item identifier beginning with 'Q'
            wikidata_id = wikidata_id[wikidata_id.index('Q'):]
            
            # API call to wikidata to get the "countries of citizenship" for this item
            request = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{wikidata_id}/statements?property=P27')
            if len(dict(request.json())) == 0:
                request = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{wikidata_id}/statements?property=P495')
            
            # create blank list to store nationalities
            nationalities = pd.Series(dtype=str)
            
            try:
                # iterate through all countries of citizenship for the composer
                for result in dict(request.json())['P27']:
                    nationalities = nationalities.append(pd.Series([result['value']['content']]), ignore_index=True)
            except KeyError:
                try:
                    for result in dict(request.json())['P495']:
                        nationalities = nationalities.append(pd.Series([result['value']['content']]), ignore_index=True)
                except KeyError:
                    nationalities = nationalities.append(pd.Series(['Q223050']), ignore_index=True)         # stateless
                
            for i in nationalities.index:
                nationalities.loc[i] = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{nationalities[i]}/labels/en').json()
                
            nationalities_df.at[index, 'nationalities'] = nationalities
    
    for index in nationalities_df.index:
        nationalities_df.at[index, 'nationalities'] = nationalities_df.at[index, 'nationalities'].to_string()
    
    # Create and store a CSV based on the nationality data
    nationalities_df.to_csv('path/to/nationalities_list.csv')